Load libraries and data.

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/anastasiyashabunevich/Desktop/Kaggle/ashabunevich_key.json"

In [2]:
from google.cloud import bigquery

#create 'Client' object
client = bigquery.Client()

#constract reference to 'openaq' dataset
dataset_ref = client.dataset('openaq', project='bigquery-public-data')

#API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

#construct reference to 'global_air_quality' table
table_ref = dataset_ref.table('global_air_quality')

#API request - fetch the table
table = client.get_table(table_ref)

#preview 'global_air_quality' table with five rows
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


### 1) Units of measurement

Which countries have reported pollution levels in units of "ppm"?  In the code cell below, set `first_query` to an SQL query that pulls the appropriate entries from the `country` column.

In [3]:
#query to select data with units 'ppm'
first_query = """
              SELECT DISTINCT country
              FROM `bigquery-public-data.openaq.global_air_quality`
              WHERE unit = "ppm"
              """

#set query to 10 GB
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

#API request - run query and return pandas DataFrame
first_results = first_query_job.to_dataframe()

#view top 5 rows of result
print(first_results.head())

  country
0      US
1      CL
2      AU
3      BM
4      MX


### 2) High air quality

Which pollution levels were reported to be exactly 0?  
- Set `zero_pollution_query` to select **all columns** of the rows where the `value` column is 0.
- Set `zero_pollution_results` to a pandas DataFrame containing the query results.

In [4]:
# Query to select all columns where pollution levels are exactly 0
zero_pollution_query = """
                       SELECT *
                       FROM `bigquery-public-data.openaq.global_air_quality`
                       WHERE value = 0
                       """

# Set up the query to 10 GB
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

# API request - run the query and return a pandas DataFrame
zero_pollution_results = query_job.to_dataframe() 

#print result 
print(zero_pollution_results.head())

                                        location     city country pollutant  \
0                     Victoria Memorial - WBSPCB  Kolkata      IN      pm25   
1  Rabindra Bharati University, Kolkata - WBSPCB  Kolkata      IN       so2   
2                   Zamość ul. Hrubieszowska 69A   Zamość      PL       no2   
3                               Końskie, MOBILNA  Końskie      PL      pm10   
4                               Końskie, MOBILNA  Końskie      PL      pm25   

   value                 timestamp   unit source_name   latitude  longitude  \
0    0.0 2017-10-16 20:45:00+00:00  µg/m³        CPCB  22.572645  88.363890   
1    0.0 2017-10-28 14:30:00+00:00  µg/m³        CPCB  22.627874  88.380400   
2    0.0 2020-05-19 05:00:00+00:00  µg/m³        GIOS  50.716630  23.290247   
3    0.0 2018-12-21 13:00:00+00:00  µg/m³        GIOS  51.189526  20.408892   
4    0.0 2018-12-21 13:00:00+00:00  µg/m³        GIOS  51.189526  20.408892   

   averaged_over_in_hours  
0                    0